In [2]:
import rasterio
from rasterio.windows import Window
import numpy as np
import os
from PIL import Image

def extract_rgb_large_tiff(input_path, output_dir, filename="output_rgb.png"):
    # 1. Create directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created directory: {output_dir}")

    output_path = os.path.join(output_dir, filename)

    with rasterio.open(input_path) as src:
        # Define RGB bands (Usually 1, 2, 3 or 5, 3, 2 depending on sensor)
        # Check your sensor specs to confirm Red, Green, and Blue indices
        indexes = (1, 2, 3) 
        
        # Get dimensions
        width = src.width
        height = src.height
        
        # To handle 9.55GB, we read the image and scale it
        # If your RAM is limited (<16GB), we use a sample to find scaling values
        print("Analyzing color range for 16-bit to 8-bit conversion...")
        sample = src.read(indexes, window=Window(width//4, height//4, 1000, 1000))
        
        # Calculate min/max for scaling (using 2nd/98th percentile for better contrast)
        mins = np.percentile(sample, 2, axis=(1, 2))
        maxs = np.percentile(sample, 98, axis=(1, 2))

        print(f"Reading and processing {width}x{height} image...")
        
        # Read the full RGB stack
        # Note: If this still hits RAM limits, we would switch to 'out_shape' tiling
        rgb = src.read(indexes)
        
        rgb_8bit = []
        for i in range(3):
            # Scale 16-bit to 8-bit range [0, 255]
            band = np.clip(rgb[i], mins[i], maxs[i])
            band = ((band - mins[i]) / (maxs[i] - mins[i]) * 255).astype(np.uint8)
            rgb_8bit.append(band)

        # Stack into HxWx3
        final_image = np.stack(rgb_8bit, axis=-1)
        
        # Save using PIL
        img = Image.fromarray(final_image)
        img.save(output_path)
        print(f"Success! Image saved at: {output_path}")

# --- Execution ---
#input_tiff = "F:/Mason/Data For Ali/20250710/Canada07102025ortho.tif" # Change this to your file path
input_tiff = "F:/Mason/Data For Ali/6-19 bean upper.tif" # Change this to your file path

save_directory = "F:/Mason/Data For Ali/20240618"         # The folder you want to save in

extract_rgb_large_tiff(input_tiff, save_directory)

Analyzing color range for 16-bit to 8-bit conversion...
Reading and processing 31606x36791 image...
Success! Image saved at: F:/Mason/Data For Ali/20240618\output_rgb.png
